In [7]:
import bioread
import pandas as pd
acq_file = bioread.read_file(r'C:\Users\User\Desktop\TIM.acq')
channels_to_extract = ['Heart Rate', 'ECG - ECG100C']
data = {}
for channel_name in channels_to_extract:
    channel = acq_file.named_channels[channel_name]
    data[channel_name] = channel.data

df = pd.DataFrame(data)
df.head()

,Heart Rate,ECG - ECG100C
0,0.0,0.118256
1,0.0,0.119476
2,0.0,0.119934
3,0.0,0.121307
4,0.0,0.121460
